In [209]:
import pandas as pd
import ast

skill = pd.read_csv('skills-2.csv')
achievement = pd.read_csv('achievements-2.csv')
role_2 = pd.read_csv('roles-2.csv')
occupation_group = pd.read_csv('occupation_group.csv')
role_2 = role_2.merge(occupation_group, left_on='occupation', right_on='occupation_name', how='left')
final_id = pd.read_csv('target_employees-2.csv')
filtered_role = role_2[role_2['anon_user_id'].isin(final_id['anon_user_id'])]
filtered_role = filtered_role[filtered_role['role_end_date'].isna()]

In [210]:
filtered_role.anon_user_id.nunique()

519

In [211]:
skill_1 = skill[skill['skill_master_id'].notna()]
unique_skill = skill_1[['anon_user_id', 'skill']].drop_duplicates()
unique_skill = unique_skill.rename(columns={'skill': 'value'})

# Assuming 'created_by' and 'achievement' columns exist in the 'achievement' DataFrame
achievement_1 = achievement[achievement['experience_master_id'].notna()]
unique_achievement = achievement_1[['anon_user_id', 'achievement']].drop_duplicates()
unique_achievement = unique_achievement.rename(columns={'achievement': 'value'})

# Concatenate the results
concatenated_df = pd.concat([unique_skill, unique_achievement], ignore_index=True)

#filtered_role = role_2[role_2['anon_user_id'].isin(final_id['anon_user_id'])]
filtered_concatenated_df = concatenated_df[concatenated_df['anon_user_id'].isin(final_id['anon_user_id'])]

skill_cate = pd.read_csv('skill_cate.csv')

merged_df = pd.merge(filtered_concatenated_df, skill_cate, left_on='value', right_on='value', how='left')

# Group by 'cate' and count unique 'created_by'
result = merged_df.groupby('cate')['anon_user_id'].nunique().reset_index()

# Rename the column for clarity
result = result.rename(columns={'anon_user_id': 'unique_people_count'})

result.to_csv('cate_distribution.csv')

merged_df = merged_df.merge(filtered_role[['anon_user_id', 'occupation_name', 'occupation_group_name']], 
                                  on='anon_user_id', how='left')

merged_df = pd.merge(merged_df, final_id, on='anon_user_id', how='inner')



In [212]:
occupation_mapping = {
    'Software QA Engineer / Tester': 'Testing & Validation',
    'QA Manager': 'Testing & Validation',
    'Validation Engineer': 'Testing & Validation',
    'Embedded Systems Engineers': 'Embedded System',
    'Software Architect': 'Architecture',
    'Systems Architect': 'Architecture',
    'Machine Learning Engineers': 'Machine Learning & Data',
    'Data Scientist': 'Machine Learning & Data'
}

# Apply the mapping to the 'occupation_name' column to create a new column 'job_cate'
merged_df['job_cate'] = merged_df['occupation_name'].map(occupation_mapping)

# If there are occupation names that don't match the mapping, you can handle them by filling with 'Unknown' or another category
merged_df['job_cate'] = merged_df['job_cate'].fillna('General Software Dev')

merged_df.loc[merged_df['cate'] == 'Embedded Systems', 'job_cate'] = 'Embedded System'


In [213]:
distinct_user_count = merged_df.groupby('job_cate')['anon_user_id'].nunique().reset_index(name='distinct_user_count')

# Step 2: Calculate the total number of distinct users across all job_cates
total_users = filtered_role['anon_user_id'].nunique()

# Step 3: Calculate the percentage of each job_cate
distinct_user_count['percentage'] = (distinct_user_count['distinct_user_count'] / total_users)

# Display the result
print(distinct_user_count)

distinct_user_count.to_csv('woven_job_cate.csv')

                  job_cate  distinct_user_count  percentage
0             Architecture                   12    0.023121
1          Embedded System                  121    0.233141
2     General Software Dev                  429    0.826590
3  Machine Learning & Data                   34    0.065511
4     Testing & Validation                   22    0.042389


top skill for country/employ type

In [214]:
location_to_country = {
    'Nihonbashi (Head Office)': 'Japan',
    'Palo Alto': 'USA',
    'Ann Arbor': 'USA',
    'London': 'United Kingdom',
    'Sunnyvale': 'USA',
    'Remote California': 'USA',
    'Remote United Kingdom': 'United Kingdom',
    'Mountain View': 'USA',
    'Remote Massachusetts': 'USA',
    'Higashifuji': 'Japan',
    'Remote Japan': 'Japan',
    'Seattle': 'USA',
    'Remote New York': 'USA',
    'Remote Oregon': 'USA',
    'Remote Washington': 'USA',
    'Brooklyn': 'USA',
    'Remote Colorado': 'USA',
    'Toyota Higashifuji': 'Japan',
    'Remote Illinois': 'USA',
    'Remote New Jersey': 'USA',
    'Remote Utah': 'USA',
    'Remote North Carolina': 'USA',
    'Remote Maryland': 'USA',
    'Remote Texas': 'USA',
    'Remote Nevada': 'USA'
}

# Assuming your DataFrame is called df and has a 'location' column
merged_df['country'] = merged_df['location'].map(location_to_country)

skill_df = pd.merge(skill_1, final_id, on='anon_user_id', how='inner')
skill_df['country'] = skill_df['location'].map(location_to_country)

country skill cate

In [215]:
cate_user_counts = merged_df.groupby(['country', 'cate'])['anon_user_id'].nunique().reset_index(name='cate_user_count')

# Calculate the total unique count of users under each job_cate
total_user_counts = merged_df.groupby('country')['anon_user_id'].nunique().reset_index(name='total_user_count')

# Merge the two dataframes to calculate the percentage
merged_counts = pd.merge(cate_user_counts, total_user_counts, on='country')

# Calculate the percentage of users in each cate under each job_cate
merged_counts['percentage'] = merged_counts['cate_user_count'] / merged_counts['total_user_count']

# Display the final result
print(merged_counts[['country', 'cate', 'cate_user_count', 'total_user_count', 'percentage']])

merged_counts.to_csv('woven_country_skill_cate.csv')

           country                                  cate  cate_user_count  \
0            Japan           Electronics & Power Systems               11   
1            Japan                      Embedded Systems               76   
2            Japan                 Machine Learning & AI               50   
3            Japan  Manufacturing & Process Optimization               19   
4            Japan            Robotics & Control Systems               10   
5            Japan                 Simulation & Modeling               31   
6              USA           Electronics & Power Systems               15   
7              USA                      Embedded Systems               38   
8              USA                   Infotainment System                1   
9              USA                 Machine Learning & AI               53   
10             USA  Manufacturing & Process Optimization               27   
11             USA            Robotics & Control Systems               18   

employee type skill cate

In [216]:
cate_user_counts = merged_df.groupby(['employee_type', 'cate'])['anon_user_id'].nunique().reset_index(name='cate_user_count')

# Calculate the total unique count of users under each job_cate
total_user_counts = merged_df.groupby('employee_type')['anon_user_id'].nunique().reset_index(name='total_user_count')

# Merge the two dataframes to calculate the percentage
merged_counts = pd.merge(cate_user_counts, total_user_counts, on='employee_type')

# Calculate the percentage of users in each cate under each job_cate
merged_counts['percentage'] = merged_counts['cate_user_count'] / merged_counts['total_user_count']

# Display the final result
print(merged_counts[['employee_type', 'cate', 'cate_user_count', 'total_user_count', 'percentage']])

merged_counts.to_csv('woven_employee_type_skill_cate.csv')

          employee_type                                  cate  \
0             Full Time           Electronics & Power Systems   
1             Full Time                      Embedded Systems   
2             Full Time                   Infotainment System   
3             Full Time                 Machine Learning & AI   
4             Full Time  Manufacturing & Process Optimization   
5             Full Time            Robotics & Control Systems   
6             Full Time                 Simulation & Modeling   
7              Secondee           Electronics & Power Systems   
8              Secondee                      Embedded Systems   
9              Secondee                 Machine Learning & AI   
10             Secondee  Manufacturing & Process Optimization   
11             Secondee            Robotics & Control Systems   
12             Secondee                 Simulation & Modeling   
13  Secondee Concurrent                      Embedded Systems   
14    Secondee Outbound  

country top skill

In [217]:
unique_counts = skill_df.groupby(['country', 'skill'])['anon_user_id'].nunique().reset_index(name='unique_user_count')

# Step 2: Filter for skills with more than 5 unique anon_user_id
filtered_skills = unique_counts[unique_counts['unique_user_count'] > 5]

# Step 3: Calculate the average skill_level for these filtered skills
average_skill_levels = skill_df.groupby(['country', 'skill'])['skill_level'].mean().reset_index(name='average_skill_level')

# Step 4: Calculate total unique users per country
total_unique_users_per_country = skill_df.groupby('country')['anon_user_id'].nunique().reset_index(name='total_user_count')

# Step 5: Merge filtered skills with total user counts to calculate percentage
filtered_skills_with_total = pd.merge(filtered_skills, total_unique_users_per_country, on='country')

# Step 6: Calculate the percentage of unique users for each skill relative to the total count of the country
filtered_skills_with_total['percentage'] = (filtered_skills_with_total['unique_user_count'] / filtered_skills_with_total['total_user_count'])

# Step 7: Merge to get the average skill level for filtered skills
merged = pd.merge(filtered_skills_with_total, average_skill_levels, on=['country', 'skill'])

# Step 8: Sort and select the top 20 skills for each country based on average skill level
top_skills = merged.sort_values(by=['country', 'average_skill_level'], ascending=[True, False])
top_20_per_country = top_skills.groupby('country').head(20)

# Display the result
print(top_20_per_country)

# Save the result to a CSV file
top_20_per_country.to_csv('top_20_per_country.csv', index=False)

            country                                              skill  \
43            Japan                       Product Lifecycle Management   
72            Japan                                      Visual Design   
4             Japan                         Artificial Neural Networks   
10            Japan                          C# (Programming Language)   
31            Japan                                 Interaction Design   
66            Japan                                       Unity Engine   
15            Japan                     Cross-Functional Collaboration   
7             Japan                                   Business Process   
30            Japan                                 Information Design   
54            Japan                                           Simulink   
74            Japan                                Workflow Management   
61            Japan                     Systems Development Life Cycle   
8             Japan                   

In [218]:
unique_counts = skill_df.groupby(['country', 'skill'])['anon_user_id'].nunique().reset_index(name='unique_user_count')

# Step 2: Filter for skills with more than 5 unique anon_user_id
filtered_skills = unique_counts[unique_counts['unique_user_count'] > 5]

# Step 3: Calculate the average skill_level for these filtered skills
average_skill_levels = skill_df.groupby(['country', 'skill'])['skill_level'].mean().reset_index(name='average_skill_level')

# Step 4: Calculate total unique users per country
total_unique_users_per_country = skill_df.groupby('country')['anon_user_id'].nunique().reset_index(name='total_user_count')

# Step 5: Merge filtered skills with total user counts to calculate percentage
filtered_skills_with_total = pd.merge(filtered_skills, total_unique_users_per_country, on='country')

# Step 6: Calculate the percentage of unique users for each skill relative to the total count of the country
filtered_skills_with_total['percentage'] = (filtered_skills_with_total['unique_user_count'] / filtered_skills_with_total['total_user_count'])

# Step 7: Merge to get the average skill level for filtered skills
merged = pd.merge(filtered_skills_with_total, average_skill_levels, on=['country', 'skill'])

# Step 8: Sort and select the top 20 skills for each country based on average skill level
top_skills = merged.sort_values(by=['country', 'average_skill_level'], ascending=[True,True])
top_20_per_country = top_skills.groupby('country').head(20)

# Display the result
print(top_20_per_country)

# Save the result to a CSV file
top_20_per_country.to_csv('bottom_20_per_country.csv', index=False)

            country                                     skill  \
24            Japan                         Functional Safety   
33            Japan                                   Jenkins   
21            Japan                                    Docker   
16            Japan                        Data Visualization   
28            Japan                                 ISO 26262   
2             Japan                 Amazon Web Services (AWS)   
36            Japan                  Manufacturing Automation   
23            Japan                   Figma (Design Software)   
50            Japan                        Quality Management   
52            Japan             Scikit-Learn (Python Package)   
55            Japan                           Software Design   
14            Japan                     Computer-Aided Design   
39            Japan                       Performance Testing   
12            Japan                                     CI/CD   
37            Japan      

In [219]:
unique_counts = skill_df.groupby(['country', 'skill'])['anon_user_id'].nunique().reset_index(name='unique_user_count')
filtered_skills = unique_counts[unique_counts['unique_user_count'] >= 10]

# Step 3: Calculate the average skill_level for these filtered skills
average_skill_levels = skill_df.groupby(['country', 'skill'])['skill_level'].mean().reset_index(name='average_skill_level')

# Step 4: Calculate total unique users per country
total_unique_users_per_country = skill_df.groupby('country')['anon_user_id'].nunique().reset_index(name='total_user_count')

# Step 5: Merge filtered skills with total user counts to calculate percentage
filtered_skills_with_total = pd.merge(filtered_skills, total_unique_users_per_country, on='country')

# Step 6: Calculate the percentage of unique users for each skill relative to the total count of the country
filtered_skills_with_total['percentage'] = (filtered_skills_with_total['unique_user_count'] / filtered_skills_with_total['total_user_count'])

# Step 7: Merge to get the average skill level for filtered skills
merged = pd.merge(filtered_skills_with_total, average_skill_levels, on=['country', 'skill'])

# Step 8: Sort and select the top 20 skills for each country based on average skill level
top_skills = merged.sort_values(by=['country', 'average_skill_level'], ascending=[True, False])
top_20_per_country = top_skills.groupby('country').head(20)

# Display the result
print(top_20_per_country)

# Save the result to a CSV file
top_20_per_country.to_csv('top_20_per_country_10.csv', index=False)

           country                                    skill  \
10           Japan           Cross-Functional Collaboration   
4            Japan                         Business Process   
50           Japan                      Workflow Management   
5            Japan                    Business Requirements   
26           Japan                      Process Improvement   
20           Japan                        Image Recognition   
19           Japan                    Human-Centered Design   
45           Japan        User Experience (UX) Design Tools   
18           Japan                                   GitHub   
13           Japan                       Design Prototyping   
38           Japan          Software Development Life Cycle   
49           Japan                              Wireframing   
39           Japan                          Sprint Planning   
24           Japan                                   MATLAB   
12           Japan                            Deep Lear

In [220]:
unique_counts = skill_df.groupby(['country', 'skill'])['anon_user_id'].nunique().reset_index(name='unique_user_count')
filtered_skills = unique_counts[unique_counts['unique_user_count'] >= 20]

# Step 3: Calculate the average skill_level for these filtered skills
average_skill_levels = skill_df.groupby(['country', 'skill'])['skill_level'].mean().reset_index(name='average_skill_level')

# Step 4: Calculate total unique users per country
total_unique_users_per_country = skill_df.groupby('country')['anon_user_id'].nunique().reset_index(name='total_user_count')

# Step 5: Merge filtered skills with total user counts to calculate percentage
filtered_skills_with_total = pd.merge(filtered_skills, total_unique_users_per_country, on='country')

# Step 6: Calculate the percentage of unique users for each skill relative to the total count of the country
filtered_skills_with_total['percentage'] = (filtered_skills_with_total['unique_user_count'] / filtered_skills_with_total['total_user_count'])

# Step 7: Merge to get the average skill level for filtered skills
merged = pd.merge(filtered_skills_with_total, average_skill_levels, on=['country', 'skill'])

# Step 8: Sort and select the top 20 skills for each country based on average skill level
top_skills = merged.sort_values(by=['country', 'average_skill_level'], ascending=[True, False])
top_20_per_country = top_skills.groupby('country').head(20)

# Display the result
print(top_20_per_country)

# Save the result to a CSV file
top_20_per_country.to_csv('top_20_per_country_20.csv', index=False)

   country                                    skill  unique_user_count  \
7    Japan           Cross-Functional Collaboration                 38   
15   Japan                      Process Improvement                 31   
26   Japan          Software Development Life Cycle                 22   
27   Japan                          Sprint Planning                 22   
9    Japan                            Deep Learning                 20   
28   Japan                            Test Planning                 20   
25   Japan                     Software Development                111   
16   Japan                        Product Execution                 33   
19   Japan                         Product Strategy                 33   
21   Japan       PyTorch (Machine Learning Library)                 26   
18   Japan                         Product Roadmaps                 31   
8    Japan                                Debugging                 67   
20   Japan                         Pro

country+common skill

In [221]:
item_counts = merged_df.groupby(['country', 'value'])['anon_user_id'].nunique().reset_index(name='count')

# Step 2: Calculate the total count of 'anon_user_id' for each 'job_cate'
total_user_counts =  merged_df.groupby('country')['anon_user_id'].nunique().reset_index(name='total_user_count')

# Step 3: Calculate the percentage for each 'value' under each 'job_cate'

merged_counts = pd.merge(item_counts, total_user_counts, on='country')

merged_counts['percentage'] = merged_counts['count'] / merged_counts['total_user_count']

# Step 4: Sort by 'job_cate' and 'count' (descending), and keep the top N items for each 'job_cate'
most_frequent_items = merged_counts.sort_values(['country', 'count'], ascending=[True, False])

# Step 5: Select the top N items for each 'job_cate' (N=20 in this case)
top_n_items_per_cate = most_frequent_items.groupby('country').head(30)

# Output the result
print(top_n_items_per_cate)

top_n_items_per_cate.to_csv('woven_country_commonskill.csv')




             country                                        value  count  \
333            Japan                                       Python    128   
393            Japan                         Software Development    111   
23             Japan                   Agile Software Development    108   
55             Japan                                          C++     92   
169            Japan                                          Git     91   
...              ...                                          ...    ...   
1010  United Kingdom                         Database Development      4   
1072  United Kingdom                            Product Execution      4   
1078  United Kingdom                             Project Planning      4   
1086  United Kingdom                          Root Cause Analysis      4   
1101  United Kingdom  Software Testing And Quality Assurance (QA)      4   

      total_user_count  percentage  
333                301    0.425249  
393          

employee type top skill

In [222]:
unique_counts = skill_df.groupby(['employee_type', 'skill'])['anon_user_id'].nunique().reset_index(name='unique_user_count')

# Step 2: Filter for skills with more than 5 unique anon_user_id
filtered_skills = unique_counts[unique_counts['unique_user_count'] > 5]

# Step 3: Calculate the average skill_level for these filtered skills
average_skill_levels = skill_df.groupby(['employee_type', 'skill'])['skill_level'].mean().reset_index(name='average_skill_level')

# Step 4: Calculate total unique users per country
total_unique_users_per_country = skill_df.groupby('employee_type')['anon_user_id'].nunique().reset_index(name='total_user_count')

# Step 5: Merge filtered skills with total user counts to calculate percentage
filtered_skills_with_total = pd.merge(filtered_skills, total_unique_users_per_country, on='employee_type')

# Step 6: Calculate the percentage of unique users for each skill relative to the total count of the country
filtered_skills_with_total['percentage'] = (filtered_skills_with_total['unique_user_count'] / filtered_skills_with_total['total_user_count'])

# Step 7: Merge to get the average skill level for filtered skills
merged = pd.merge(filtered_skills_with_total, average_skill_levels, on=['employee_type', 'skill'])

# Step 8: Sort and select the top 20 skills for each country based on average skill level
top_skills = merged.sort_values(by=['employee_type', 'average_skill_level'], ascending=[True, False])
top_20_per_country = top_skills.groupby('employee_type').head(20)

# Display the result
print(top_20_per_country)

# Save the result to a CSV file
top_20_per_country.to_csv('top_20_per_employeetype.csv', index=False)

    employee_type                                              skill  \
30      Full Time                                         Hand Tools   
61      Full Time  SLAM Algorithms (Simultaneous Localization And...   
4       Full Time                         Artificial Neural Networks   
24      Full Time                                  Electrical Wiring   
64      Full Time                                    Software Design   
22      Full Time                                 Design Prototyping   
16      Full Time                     Cross-Functional Collaboration   
33      Full Time                                 Interaction Design   
15      Full Time                                     Control Theory   
31      Full Time                              Human-Centered Design   
70      Full Time                               Systems Architecture   
47      Full Time                               Power Tool Operation   
32      Full Time                                  Image Recogni

employee type+bottom skill

In [223]:
unique_counts = skill_df.groupby(['employee_type', 'skill'])['anon_user_id'].nunique().reset_index(name='unique_user_count')

# Step 2: Filter for skills with more than 5 unique anon_user_id
filtered_skills = unique_counts[unique_counts['unique_user_count'] > 5]

# Step 3: Calculate the average skill_level for these filtered skills
average_skill_levels = skill_df.groupby(['employee_type', 'skill'])['skill_level'].mean().reset_index(name='average_skill_level')

# Step 4: Calculate total unique users per country
total_unique_users_per_country = skill_df.groupby('employee_type')['anon_user_id'].nunique().reset_index(name='total_user_count')

# Step 5: Merge filtered skills with total user counts to calculate percentage
filtered_skills_with_total = pd.merge(filtered_skills, total_unique_users_per_country, on='employee_type')

# Step 6: Calculate the percentage of unique users for each skill relative to the total count of the country
filtered_skills_with_total['percentage'] = (filtered_skills_with_total['unique_user_count'] / filtered_skills_with_total['total_user_count'])

# Step 7: Merge to get the average skill level for filtered skills
merged = pd.merge(filtered_skills_with_total, average_skill_levels, on=['employee_type', 'skill'])

# Step 8: Sort and select the top 20 skills for each country based on average skill level
top_skills = merged.sort_values(by=['employee_type', 'average_skill_level'], ascending=[True,True])
top_20_per_country = top_skills.groupby('employee_type').head(20)

# Display the result
print(top_20_per_country)

# Save the result to a CSV file
top_20_per_country.to_csv('bottom_20_per_employeetype.csv', index=False)

    employee_type                                     skill  \
41      Full Time                  Manufacturing Automation   
36      Full Time                                   Jenkins   
77      Full Time                                   Tooling   
46      Full Time                                PostgreSQL   
60      Full Time               Rust (Programming Language)   
21      Full Time  Design Failure Mode And Effects Analysis   
74      Full Time                           Test Automation   
38      Full Time                                    MATLAB   
87      Full Time                     Workshop Facilitation   
2       Full Time                 Amazon Web Services (AWS)   
57      Full Time                        Quality Management   
23      Full Time                                    Docker   
34      Full Time               Java (Programming Language)   
35      Full Time         JavaScript (Programming Language)   
73      Full Time                                 Terra

woven job cate skill cate cross analysis - don't review

In [224]:
cate_user_counts = merged_df.groupby(['job_cate', 'cate'])['anon_user_id'].nunique().reset_index(name='cate_user_count')

# Calculate the total unique count of users under each job_cate
total_user_counts = merged_df.groupby('job_cate')['anon_user_id'].nunique().reset_index(name='total_user_count')

# Merge the two dataframes to calculate the percentage
merged_counts = pd.merge(cate_user_counts, total_user_counts, on='job_cate')

# Calculate the percentage of users in each cate under each job_cate
merged_counts['percentage'] = merged_counts['cate_user_count'] / merged_counts['total_user_count']

# Display the final result
print(merged_counts[['job_cate', 'cate', 'cate_user_count', 'total_user_count', 'percentage']])

merged_counts.to_csv('woven_job_cate_skill_cate.csv')

                   job_cate                                  cate  \
0              Architecture           Electronics & Power Systems   
1              Architecture                 Machine Learning & AI   
2              Architecture  Manufacturing & Process Optimization   
3              Architecture                 Simulation & Modeling   
4           Embedded System           Electronics & Power Systems   
5           Embedded System                      Embedded Systems   
6           Embedded System                   Infotainment System   
7           Embedded System                 Machine Learning & AI   
8           Embedded System            Robotics & Control Systems   
9           Embedded System                 Simulation & Modeling   
10     General Software Dev           Electronics & Power Systems   
11     General Software Dev                   Infotainment System   
12     General Software Dev                 Machine Learning & AI   
13     General Software Dev  Manuf

job cate + common skill

In [225]:
# Step 1: Group by 'job_cate' and 'value' to get the unique count of 'anon_user_id'
item_counts = merged_df.groupby(['job_cate', 'value'])['anon_user_id'].nunique().reset_index(name='count')

# Step 2: Calculate the total count of 'anon_user_id' for each 'job_cate'
total_user_counts = merged_df.groupby('job_cate')['anon_user_id'].nunique().reset_index(name='total_user_count')

# Step 3: Calculate the percentage for each 'value' under each 'job_cate'
merged_counts = pd.merge(item_counts, total_user_counts, on='job_cate')

merged_counts['percentage'] = merged_counts['count'] / merged_counts['total_user_count']

# Step 4: Sort by 'job_cate' and 'count' (descending), and keep the top N items for each 'job_cate'
most_frequent_items = merged_counts.sort_values(['job_cate', 'count'], ascending=[True, False])

# Step 5: Select the top N items for each 'job_cate' (N=20 in this case)
top_n_items_per_cate = most_frequent_items.groupby('job_cate').head(20)

# Output the result
print(top_n_items_per_cate)

top_n_items_per_cate.to_csv('woven_jobcate_commonskill.csv')


                  job_cate                            value  count  \
44            Architecture                            Linux      9   
72            Architecture     Software Architecture Design      8   
1             Architecture                             ADAS      7   
73            Architecture             Software Development      7   
11            Architecture                              C++      6   
...                    ...                              ...    ...   
928   Testing & Validation                Agile Methodology      6   
990   Testing & Validation  Software Development Life Cycle      6   
936   Testing & Validation           Continuous Integration      5   
982   Testing & Validation              Root Cause Analysis      5   
1003  Testing & Validation      Testing Scenario Management      5   

      total_user_count  percentage  
44                  12    0.750000  
72                  12    0.666667  
1                   12    0.583333  
73         

country+job_cate

In [226]:
# Step 1: Group by 'job_cate' and 'value' to get the unique count of 'anon_user_id'


item_counts = merged_df.groupby(['country', 'job_cate'])['anon_user_id'].nunique().reset_index(name='count')

# Step 2: Calculate the total count of 'anon_user_id' for each 'job_cate'
total_user_counts = merged_df.groupby('country')['anon_user_id'].nunique().reset_index(name='total_user_count')

# Step 3: Calculate the percentage for each 'value' under each 'job_cate'
merged_counts = pd.merge(item_counts, total_user_counts, on='country')

merged_counts['percentage'] = merged_counts['count'] / merged_counts['total_user_count']

print(merged_counts)

merged_counts.to_csv('woven_country_jobcate.csv')

           country                 job_cate  count  total_user_count  \
0            Japan             Architecture      9               301   
1            Japan          Embedded System     80               301   
2            Japan     General Software Dev    241               301   
3            Japan  Machine Learning & Data     23               301   
4            Japan     Testing & Validation     11               301   
5              USA             Architecture      3               194   
6              USA          Embedded System     38               194   
7              USA     General Software Dev    170               194   
8              USA  Machine Learning & Data      9               194   
9              USA     Testing & Validation     11               194   
10  United Kingdom          Embedded System      3                20   
11  United Kingdom     General Software Dev     18                20   
12  United Kingdom  Machine Learning & Data      2              

employee type + job_category

In [227]:
# Step 1: Group by 'job_cate' and 'value' to get the unique count of 'anon_user_id'
item_counts = merged_df.groupby(['employee_type', 'job_cate'])['anon_user_id'].nunique().reset_index(name='count')

# Step 2: Calculate the total count of 'anon_user_id' for each 'job_cate'
total_user_counts = merged_df.groupby('employee_type')['anon_user_id'].nunique().reset_index(name='total_user_count')

# Step 3: Calculate the percentage for each 'value' under each 'job_cate'
merged_counts = pd.merge(item_counts, total_user_counts, on='employee_type')

merged_counts['percentage'] = merged_counts['count'] / merged_counts['total_user_count']

print(merged_counts)

                      employee_type                 job_cate  count  \
0                        Fixed Term     General Software Dev      2   
1                         Full Time             Architecture      8   
2                         Full Time          Embedded System     70   
3                         Full Time     General Software Dev    263   
4                         Full Time  Machine Learning & Data     22   
5                         Full Time     Testing & Validation     14   
6                          Secondee             Architecture      4   
7                          Secondee          Embedded System     49   
8                          Secondee     General Software Dev    158   
9                          Secondee  Machine Learning & Data     12   
10                         Secondee     Testing & Validation      8   
11              Secondee Concurrent          Embedded System      1   
12              Secondee Concurrent     General Software Dev      2   
13  Se

similar people we have

In [228]:
max_experience = achievement_1.groupby('anon_user_id')['year_of_experiences'].max().reset_index()

# Step 2: Filter for max year_of_experiences >= 5
filtered_experience = max_experience[max_experience['year_of_experiences'] >= 5]

# Step 3: Get the unique list of 'created_by'
unique_created_by = filtered_experience['anon_user_id'].unique()

filtered_df2 = filtered_concatenated_df[filtered_concatenated_df['anon_user_id'].isin(unique_created_by)]

# Step 2: List of values to check for
target_values = [
    'Embedded System Configuration', 
    'Sensor Systems', 
    'Microcontrollers', 
    'Real-Time Control Firmwares', 
    'Process Failure Mode And Effects Analysis (PFMEA)', 
    'Design Failure Mode And Effects Analysis'
]

# Step 3: Filter rows where 'value' is in the target values
filtered_values_df = filtered_df2[filtered_df2['value'].isin(target_values)]

# Step 4: Group by 'created_by' and count the number of unique target values for each person
value_counts = filtered_values_df.groupby('anon_user_id')['value'].nunique().reset_index()

# Step 5: Filter for those who have covered at least 4 values
sufficient_coverage = value_counts[value_counts['value'] >= 3]

# Step 6: Get the count of created_by who meet the criteria
count_created_by = sufficient_coverage['anon_user_id'].nunique()

# Display the result
print(f"Number of 'created_by' with at least 4 covered values: {count_created_by}")

Number of 'created_by' with at least 4 covered values: 0


industry analysis

In [229]:
filtered_df = pd.read_csv('filtered_engineer.csv')

In [237]:
filtered_df.groupby(['job_company_name'])['full_name'].nunique()

job_company_name
cariad              245
cruise               47
horizon robotics     13
momenta               6
motional             29
tesla               108
waymo                23
wayve                 7
zoox                 44
Name: full_name, dtype: int64

In [ ]:
filtered_df.

create job level

In [230]:
def categorize_job_level(job_title):
    job_title = job_title.lower()
    if 'manager' in job_title:
        return 'L9'
    elif 'tech lead' in job_title:
        return 'L9'
    elif 'senior staff' in job_title:
        return 'L7'
    elif 'staff' in job_title:
        return 'L6'
    elif 'principal' in job_title:
        return 'L8'
    elif 'senior' in job_title:
        return 'L5'
    else:
        return 'Other'

# Apply the functions to create new columns

filtered_df['job_level'] = filtered_df['job_title'].apply(categorize_job_level)

def categorize_job_level2(job_level):
    #job_level = job_level.lower()
    if job_level == 'L5' or job_level =='L6':
        return 'L5-6'
    elif job_level == 'L7' or job_level =='L8':
        return 'L7-8'
    elif job_level == 'Other':
        return 'General'
    else:
        return 'L9'

# Apply the functions to create new columns

filtered_df['job_level2'] = filtered_df['job_level'].apply(categorize_job_level2)

job cate

In [231]:
ml_data_keywords = ['data scien', 'data anal', 'machine lear', 'algorithm']
testing_keywords = ['test', 'validation', 'quality', 'qa']
safety_keywords = ['safety', 'reliability']
function_concept_keywords = ['function and concept']
architect_keywords = ['architect']

embedded_system_related_keywords = [
    "Embedded", "Sensor", "Sensors", "Sensorset", "Electronic Control Units", 
    "Microcontroller", "Microprocessor", "Actuator", "Radar", 
    "LiDAR", "Ultrasonic Sensor", "Camera", "Real-time operating system",
    "Communication Interface", "CAN Bus", "Ethernet", 
    "FlexRay", "Power Management", "control system","ecu","Controller Area Network","rtos"
]

embedded_system_related_keywords = [keyword.lower() for keyword in embedded_system_related_keywords]

# Function to categorize job titles based on keywords
def categorize_job(row):
    job_title = row['job_title'].lower()
    summary_achievements = row['summary_achievements'].lower()  # Join list to string
    skills = ' '.join(row['skills']).lower()  # Join list to string

    # Machine Learning and Data related
    if any(keyword in job_title for keyword in ml_data_keywords):
        return 'Machine Learning & Data'
    
    # Testing & Validation
    elif any(keyword in job_title for keyword in testing_keywords):
        return 'Testing & Validation'
    
    # Safety
    elif any(keyword in job_title for keyword in safety_keywords):
        return 'Functional Safety'
    
    # Function and concept engineer
    elif any(keyword in job_title for keyword in function_concept_keywords):
        return 'Function & concept'
    
    # Systems Architecture related
    elif any(keyword in job_title for keyword in architect_keywords):
        return 'Architecture'
    
    # Embedded System related
    elif any(keyword in summary_achievements for keyword in embedded_system_related_keywords) or \
         any(keyword in skills for keyword in embedded_system_related_keywords):
        return 'Embedded System'
    
    # General Software Engineer (default)
    else:
        return 'General Software Dev'

# Apply the function to create the new column
filtered_df['job_cate'] = filtered_df.apply(categorize_job, axis=1)

core skill count

In [232]:
exploded_df = filtered_df.explode('mapped_result')

# Step 2: Merge with skill_cate to get the category (cate) for each value in `mapped_result`
market_merged_df = exploded_df.merge(skill_cate, left_on='mapped_result', right_on='value', how='inner')
unique_counts = market_merged_df.groupby(['job_cate', 'job_level2', 'full_name'])['mapped_result'].nunique().reset_index(name='unique_mapped_count')

# Step 2: Calculate the average unique count for each combination of 'job_cate' and 'job_level2'
average_unique_counts = unique_counts.groupby(['job_cate', 'job_level2'])['unique_mapped_count'].mean().reset_index(name='average_unique_mapped_count')

# Display the result
print(average_unique_counts)
average_unique_counts.to_csv('average_core_skill.csv')


Empty DataFrame
Columns: [job_cate, job_level2, average_unique_mapped_count]
Index: []


most common skill under each job category

In [233]:
item_counts = exploded_df.groupby(['job_cate', 'mapped_result'])['full_name'].nunique().reset_index(name='count')

# Step 3: Find the most frequent items for each `job_cate`
# Sort by `job_cate` and `count` (descending), then keep only the top item for each `job_cate`
most_frequent_items = item_counts.sort_values(['job_cate', 'count'], ascending=[True, False])
total_user_counts = exploded_df.groupby('job_cate')['full_name'].nunique().reset_index(name='total_user_count')
merged_counts = pd.merge(most_frequent_items, total_user_counts, on='job_cate')
merged_counts['percentage'] = merged_counts['count'] / merged_counts['total_user_count']
# Optionally: You can keep the top N items for each job_cate using the following code
top_n_items_per_cate = merged_counts.groupby('job_cate').head(30)  # Change `1` to `N` for top N

# Output the result
print(top_n_items_per_cate)
top_n_items_per_cate.to_csv('market_job_cate_skills.csv')

                 job_cate                                      mapped_result  \
0            Architecture  ['ADAS algorithms', 'ADAS', 'Pipeline Developm...   
1            Architecture         ['ADAS algorithms', 'HIL Testing', 'ADAS']   
2            Architecture  ['ADAS', 'Digital Signal Processing', 'ADAS al...   
3            Architecture  ['Android Applications', 'JavaScript (Programm...   
4            Architecture  ['Application Lifecycle Management (ALM) Softw...   
..                    ...                                                ...   
438  Testing & Validation  ['C++', 'Eclipse (Software)', 'Semantic HTML',...   
439  Testing & Validation  ['C++', 'Equipment Design', 'Industrial Roboti...   
440  Testing & Validation  ['C++', 'Lauterbach', 'Eclipse (Software)', 'G...   
441  Testing & Validation  ['C++', 'Microsoft SQL Servers', 'Embedded Sys...   
442  Testing & Validation  ['C++', 'Semantic HTML', '3D Modeling', 'Safet...   

     count  total_user_count  percentag

education

In [234]:

filtered_df['education'] = filtered_df['education'].apply(ast.literal_eval)
grouped = filtered_df.groupby('job_cate')

# Step 2: Define a function that performs the analysis on each group
def process_group(df):
    # Explode the 'education' column
    education_exploded = df.explode('education').reset_index(drop=True)
    
    # Normalize the nested JSON structure to flatten it into columns
    education_df = pd.json_normalize(education_exploded['education'])
    
    # Add the 'full_name' back to the normalized DataFrame
    education_df['full_name'] = education_exploded['full_name']
    
    # Convert 'end_date' to numeric, handling cases where it might be missing or None
    education_df['end_date'] = pd.to_numeric(education_df['end_date'], errors='coerce')
    
    # Sort by 'full_name' and 'end_date' to get the most recent entry per person
    education_df_sorted = education_df.sort_values(by=['full_name', 'end_date'], ascending=[True, False])
    
    # Drop duplicates keeping the most recent entry per person
    education_most_recent = education_df_sorted.drop_duplicates(subset=['full_name'], keep='first')
    
    # Select the relevant columns: 'degrees', 'school.name', and 'full_name'
    education_filtered = education_most_recent[['full_name', 'degrees', 'school.name']]
    
    return education_filtered

# Step 3: Apply the function to each group and concatenate the results
education_per_job_cate = grouped.apply(process_group).reset_index(drop=True)

# Display the result
#print(education_per_job_cate)


def categorize_degree(degree):
    # Convert degree to lowercase for easier matching
    degree = str(degree).lower()
    
    if 'master' in degree:
        return 'master'
    elif 'bachelor' in degree:
        return 'bachelor'
    elif 'doctor' in degree or 'phd' in degree:
        return 'phd'
    else:
        return 'below undergrad'

# Apply the categorize_degree function to the 'degrees' column
education_per_job_cate['degree_category'] = education_per_job_cate['degrees'].apply(categorize_degree)



# Merge filtered_df and education_per_job_cate on 'full_name'
merged_df = filtered_df.merge(education_per_job_cate[['full_name', 'degree_category']], on='full_name', how='left')

# Step 1: Calculate the count of each degree_category within each job_cate and job_level2
degree_count_per_job_cate = merged_df.groupby(['job_cate', 'job_level2', 'degree_category']).size().reset_index(name='degree_count')

# Step 2: Calculate the total count of all degrees for each job_cate and job_level2
total_count_per_job_cate = merged_df.groupby(['job_cate', 'job_level2']).size().reset_index(name='total_count')

# Step 3: Merge the two DataFrames to calculate the percentage
merged_counts = pd.merge(degree_count_per_job_cate, total_count_per_job_cate, on=['job_cate', 'job_level2'])

# Step 4: Calculate the percentage of each degree_category within job_cate and job_level2
merged_counts['%_of_degrees'] = (merged_counts['degree_count'] / merged_counts['total_count']) * 100

# Display the result
print(merged_counts)

                job_cate job_level2  degree_category  degree_count  \
0           Architecture    General         bachelor             1   
1           Architecture    General  below undergrad             4   
2           Architecture    General           master             2   
3           Architecture    General              phd             2   
4           Architecture       L5-6         bachelor             2   
..                   ...        ...              ...           ...   
63  Testing & Validation       L5-6           master            13   
64  Testing & Validation       L5-6              phd             1   
65  Testing & Validation       L7-8           master             1   
66  Testing & Validation         L9         bachelor             1   
67  Testing & Validation         L9           master             1   

    total_count  %_of_degrees  
0             9     11.111111  
1             9     44.444444  
2             9     22.222222  
3             9     22.222222  

company+job cate 

In [235]:
import pandas as pd
import matplotlib.pyplot as plt

# Step 1: Group the data by 'job_company_name' and 'job_cate' to get the counts
job_cate_counts = filtered_df.groupby(['job_company_name', 'job_cate']).size().reset_index(name='count')

# Step 2: Calculate the total count per company to get percentages
job_cate_counts['total_count'] = job_cate_counts.groupby('job_company_name')['count'].transform('sum')
job_cate_counts['percentage'] = (job_cate_counts['count'] / job_cate_counts['total_count']) * 100

# Step 3: Plot pie charts for each job_company_name
companies = job_cate_counts['job_company_name'].unique()